In [1]:
import os
import random
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Tokenizer, T5Model, T5Config
import sys
sys.path.append('../')
from data.dataset import CodeSnifferDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sniffer import CodeSnifferNetwork
import torch
import torch.optim as optim


random.seed(90)

In [ ]:
#codeModel.config.n_positions = 4096
#codeTokenizer.model_max_length = 4096
#text = "def greet(user): print(f'hello <extra_id_0>!')"
#input_ids = codeTokenizer(text, return_tensors="pt").input_ids

In [ ]:
dataset = CodeSnifferDataset(annotations_file="../data/dataset_test.csv", code_files_dir="../data/code_files")

train_size = int(0.8 * len(dataset))    # 80% for training and 20% for testing.
test_size = len(dataset) - train_size
training_data, test_data = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
codeModel = CodeSnifferNetwork(num_labels=8)
# codeModel = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

criterion = torch.nn.BCELoss()
optimizer = optim.Adam(codeModel.parameters(), lr=0.00002)

In [ ]:
# Training loop
